In [ ]:
using LinearAlgebra
using JLD
using Formatting
using Plots
using LaTeXStrings

# Module containing fractal Hamiltonian functions
# Encapsulates all the quantum fractal simulation code
module func
    include("fractal_hamiltonian.jl")
end

datapath = "Specify path to the saved eigenspectra data here";

#Constants
a = 1.1;
r0 = a/4;

In [ ]:
#Loading eigen values and vectors
function ValVec(n,M,N0,V,mass)
    val = load(datapath*"Val_G$(fmt("0>1", n))_($(fmt("0>1", M)),$(fmt("0>1", N0)))_$(fmt("0>2", V))V_$(fmt("0>1", mass)).jld")["data"];
    vec = load(datapath*"Vec_G$(fmt("0>1", n))_($(fmt("0>1", M)),$(fmt("0>1", N0)))_$(fmt("0>2", V))V_$(fmt("0>1", mass)).jld")["data"];
    return val,vec
end 

#Total number of points in a G(m) scaling frame of a G(n) fractal
function ScalingSize(n,M,N0,switch)
    result = [];
    for m=1:n
        if switch == 0 #Rectangular Scaling
            i1,i2,j1,j2 = func.ScalingDim(a,m,n,M,N0)
            dim = (i2+1-i1)*(j2+1-j1)
        else #Triangular Scaling
            dim = func.TScalingDim(a,n,m,M,N0)
        end
        push!(result,sqrt(dim))
    end
    return result
end

#Ratio of elements in the list that lie in the interval [P0-delta/2,P0+delta/2] to the length of Plist
function Probability(P0,Plist::AbstractArray,delta)
    count = 0
    for i=1:length(Plist)
        if P0-delta/2 <= Plist[i] < P0+delta/2
            count += 1
        end
    end
    return count/length(Plist)
end


$\color{blue}{\Large{\textbf{Distribution of IPR}}}$

In [ ]:
#Typical log IPR value corresponding to the states in the energy interval [E1,E2]
function IPRtyp(m,n,M,N0,V,mass,q,E1,E2,switch)
    val,vec = ValVec(n,M,N0,V,mass);
    
    #List of log of IPR values in the interval [E1,E2]
    logIPR = [];
    for i=1:length(val)
        if E1 <= val[i] <= E2
            ipr = func.IPRscaled(a,val,vec,val[i],val[i],M,N0,n,m,q,switch)
            push!(logIPR,log(ipr))
        end
    end
    
    #Returns <ln IPR>
    return sum(logIPR)/length(logIPR)
end

#Distribution of Log[IPR] corresponding to the states in the energy interval [E1,E2]
function LogIPRdist(m,n,M,N0,V,mass,E1,E2,delta,switch)
    val,vec = ValVec(n,M,N0,V,mass);
    
    #List of log of IPR values in the interval [E1,E2]
    logIPR = [];
    for i=1:length(val)
        if E1 <= val[i] <= E2
            ipr = func.IPRscaled(a,val,vec,val[i],val[i],M,N0,n,m,2,switch)#q=2
            push!(logIPR,log(ipr))
        end
    end
    
    #X-axis for the probability distribution plot
    X = collect(range(minimum(logIPR),stop=maximum(logIPR),step = delta));
    
    #Y-axis for the probability distribution plot
    Y = [];
    for i=1:length(X)
        push!(Y,Probability(X[i],logIPR,delta))
    end
    
    #Returns X and Y axis for the probability distribution plot for a G(m) scaled DF
    return X,Y
end

#Log[IPR] distribution plot where different m cases are plotted separately
function plotLogIPRdist(n,M,N0,V,mass,E1,E2,delta)
    pList = [];
    colorList = ["red","blue","green","black"];
    labelList = ["m=1","m=2","m=3","m=4"]
    for m=1:n
        X,Y = LogIPRdist(m,n,M,N0,V,mass,E1,E2,delta,1); #Triangular Scaling
        p = plot(X,Y,seriestype= :scatter,color=colorList[m],label = labelList[m])
        push!(pList,p)
    end
    p = plot(pList[1],pList[2],pList[3],pList[4],layout=(2,2),dpi = 500,
        xguidefontsize=8,yguidefontsize=8,legendfontsize=5,legend=:topright,markersize=:3)
    xlabel!(L"\ln P_2")
    ylabel!(L"\mathcal{P}(\ln P_2)")
    display(p)
end  

#Normalized Log[IPR] distribution plot 
function plotLogIPRdistTyp(n,M,N0,V,mass,E1,E2,delta)
    pList = [];
    colorList = ["red","blue","green","black"];
    labelList = ["m=1","m=2","m=3","m=4"]
    XList = [];
    YList = [];
    for m=1:n
        typ = IPRtyp(m,n,M,N0,V,mass,2,E1,E2,1);#q=2;Triangular Scaling 
        X1,Y = LogIPRdist(m,n,M,N0,V,mass,E1,E2,delta,1);#Triangular Scaling
        X = [];
        for value in X1
            push!(X,value-typ)
        end
        push!(XList,X)
        push!(YList,Y)
    end
    p = plot(XList[1],YList[1],seriestype= :scatter,color=colorList[1],label = labelList[1],dpi=500)
    plot!(XList[2],YList[2],seriestype= :scatter,color=colorList[2],label = labelList[2],dpi=500)
    plot!(XList[3],YList[3],seriestype= :scatter,color=colorList[3],label = labelList[3],dpi=500)
    plot!(XList[4],YList[4],seriestype= :scatter,color=colorList[4],label = labelList[4],dpi=500)
    xlabel!(L"\ln P_2 -  <\ln P_2 >")
    ylabel!(L"\mathcal{P}(\ln P_2)")
    display(p)
end

$\color{blue}{\Large{\textbf{Scaling Exponents of IPR}}}$

In [ ]:
#Scaled IPR from Gen n for states in the interval [Ei,Ef)
function IPRGn(n,M,N0,V,mass,Ei,Ef,q,switch)
    #switch = 1 for the list of typical ln IPR
    val,vec = ValVec(n,M,N0,V,mass)
    result = [];
    for m=1:n
        if switch == 1
            ipr = exp(IPRtyp(m,n,M,N0,V,mass,q,Ei,Ef,1))#Triangular Scaling
        else
            ipr = func.IPRscaled(a,val,vec,Ei,Ef,M,N0,n,m,q,1) #Triangular Scaling
        end    
        push!(result,ipr)
    end
    return result
end

#Fractal dimension from triangular scaled IPR
function FDim(n,M,N0,V,mass,Ei,Ef,q,switch)
    #switch = 1 for the list of typical ln IPR
    systemsize = log.(ScalingSize(n,M,N0,1));#Triangular Scaling
    ipr = log.(IPRGn(n,M,N0,V,mass,Ei,Ef,q,switch));
    X = hcat(ones(length(systemsize)),systemsize)
    Y = ipr
    intercept,slope = inv(X'*X)*(X'*Y);
    D = round(-slope/(q-1),digits=3)
    error= [];
    for i=1:length(Y)
        y = slope*systemsize[i]+intercept
        push!(error,(y-ipr[i])^2)
    end
    var = round(sum(error)/length(Y),digits=6)
    scatter(systemsize, ipr, fmt = :png,legend=:false)
    p = plot!((x) -> intercept + slope * x, 2, 5)
    xlabel!(L"\ln L")
    ylabel!(L"\ln IPR")
    annotate!(4.5,-5,"D = $D",10)
    annotate!(4.5,-5.5,"variance = $var",10)
    display(p)
end

$\color{blue}{\Large{\textbf{Distribution of Scaling Exponent}}}$

In [ ]:
#Fractal dimension with given lists of log IPR and log system size
function FDimState(systemsize,IPR)
    X = hcat(ones(length(systemsize)),systemsize)
    Y = IPR
    intercept,slope = inv(X'*X)*(X'*Y);
    error= [];
    for i=1:length(Y)
        y = slope*systemsize[i]+intercept
        push!(error,(y-IPR[i])^2)
    end
    var = sum(error)/length(Y)
    return slope,var
end

##Correlation dimension corresponding to each state in the given energy interval
function DimState(n,M,N0,V,mass,E1,E2,q)
    val,vec = ValVec(n,M,N0,V,mass)
    
    #List of state indices in the energy interval [E1,E2]
    Indices = [];
    for i=1:length(val)
        if E1 <= val[i] <= E2
            push!(Indices,i)
        end
    end
    
    #System size corresponding to generations 1 to m 
    L = log.(ScalingSize(n,M,N0,1))#Triangular Scaling
    
    #Correlation dimension corresponding to each state index in Indices.
    Dimensions = [];
    for i=1:length(Indices)
        logIPR = []
        for m=1:n
            ipr = func.ScaledIPRstate(vec,a,n,m,M,N0,q,Indices[i])
            push!(logIPR,log(ipr))
        end
        slope,var = FDimState(L,logIPR)
        push!(Dimensions,-slope/(q-1))
    end
    
    #List of dimensions
    return real(Dimensions)
end

#Plot the distribution of D where D is calculated from the scaled IPR
function plotD(n,M,N0,V,mass,E1,E2,delta)
    
    #X axis
    X = collect(range(0,stop=3,step=delta))
    
    #Y axis
    Dimensions = DimState(n,M,N0,V,mass,E1,E2,2) #q=2
    Y = [];
    for i=1:length(X)
        count = 0
        for j=1:length(Dimensions)
            if X[i]-delta/2 <= Dimensions[j] < X[i] + delta/2
                count += 1
            end
        end
        push!(Y,count/length(Dimensions))
    end
    
    #Average and most probable value of D
    avg = round(sum(Dimensions)/length(Dimensions),digits=4)
    mp = round(X[argmax(Y)],digits=4)
        
    #Plotting the distribution
    p = plot(X,Y,dpi=500,line=:dash,lw=1,color=:blue,marker=Plots.supported_markers()[3],markersize=:3
        ,legend=:false)
    xlabel!(L"D_2")
    ylabel!(L"\mathcal{P}(D_2)")
    annotate!(2.6,0.1,"<D> = $avg",10)
    annotate!(mp+0.1,maximum(Y),"$mp",10)
    display(p)
    return 
end